In [7]:
import pandas as pd
import torch

In [5]:
# settings
pd.options.display.precision = 3

### utils

In [4]:
import inspect
from collections import defaultdict
from torch.utils import benchmark 

def var_dict(*args):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return dict([(name, val) for name, val in callers_local_vars if val is arg][0] 
                for arg in args)

def walltime(stmt, arg_dict, duration=3):
    return benchmark.Timer(stmt=stmt, globals=arg_dict).blocked_autorange(
        min_run_time=duration).median

### 接受 dict of dict 

- 两层字典的 key 分别所谓行列的索引；

In [10]:
from tqdm import tqdm
vector = defaultdict(lambda: {})
# *4
for n in tqdm([1024*64, 1024*256, 1024*1024, 1024*1024*4, 1024*1024*16, 1024*1024*64]):
    a = torch.randn(n).cuda()
    t = walltime('a * 1.2', var_dict(a))
    vector[n]['TFLOPS'] = n / t / 1e12
    # float32: 4 Byte;
    # 读写：两个操作；
    vector[n]['GB/s'] = (4*2) * n / t / 1e9
    
pd.DataFrame(vector)

100%|██████████| 6/6 [00:22<00:00,  3.75s/it]


,65536,262144,1048576,4194304,16777216,67108864
TFLOPS,0.011,0.045,0.180,0.473,0.115,0.115
GB/s,89.162,363.744,1438.089,3781.612,919.468,921.305
